In [2]:
library(xgboost)

In [39]:
source("../src/data/read_dmc.R")

orig_train = read_dmc("../data/train.csv")
orig_items = read_dmc("../data/items.csv")
orig_class = read_dmc("../data/class.csv")

In [12]:
df = merge(train, items, by = "pid")

df = df[order(df$lineID), ]

In [16]:
label = df$revenue
train_mat = df[!(names(df) %in% c("click", "basket", "order", "revenue"))]

In [17]:
head(train_mat)

,pid,lineID,day,adFlag,availability,competitorPrice,price,manufacturer,group,content,unit,pharmForm,genericProduct,salesIndex,category,campaignIndex,rrp
953577,6570,1,1,0,2,14.6,16.89,255,2FOI,50,ML,TRO,0,40,193,NA,18.25
1916348,14922,2,1,1,1,8.57,8.75,18,1COJ0FIK,50,ST,TAB,1,40,66,C,18.81
2109080,16382,3,1,0,1,14.77,16.06,41,22OI7,2X50,ML,STI,0,53,40,NA,18.48
129870,1145,4,1,1,1,6.59,6.55,52,18OZ00IS,60,G,GEL,0,40,25,NA,9.31
528907,3394,5,1,0,1,4.39,4.14,90,20OI0,25X2,ST,KOM,0,53,14,NA,8.13
560194,3661,6,1,0,1,13.66,10.03,90,13OX06,1000,ML,LOE,0,52,127,NA,21.6


## Meeting 4/25

Do competitor prices vary within PIDs?

In [23]:
sort(table(train$pid), decreasing = T)[1:10]


 2655  2491 16641 16144  7403  3853  1426  1247  1037  1567 
53785 14666 12369 11134 10002  8683  8396  7949  7933  7761 

In [35]:
z = aggregate(train$price, list(train$day, train$pid), var)

In [38]:
sum(z$x[!is.na(z$x)])

[1] 2446.428

The above shows there is some variation within days.

In [61]:
train = df[df$day <= 62, ]
test = df[df$day > 62, ]

In [62]:
label_cols = c("click", "basket", "order", "revenue")

train_labels = train[label_cols]
train = train[!(names(train) %in% label_cols)]

test_labels = test[label_cols]
test = test[!(names(test) %in% label_cols)]

In [63]:
delete_cols = c("pid", "lineID")
train = train[!(names(train) %in% delete_cols)]
test = test[!(names(test) %in% delete_cols)]

In [50]:
head(train)

,day,adFlag,availability,competitorPrice,price,manufacturer,group,content,unit,pharmForm,genericProduct,salesIndex,category,campaignIndex,rrp
953577,1,0,2,14.6,16.89,255,2FOI,50,ML,TRO,0,40,193,NA,18.25
1916348,1,1,1,8.57,8.75,18,1COJ0FIK,50,ST,TAB,1,40,66,C,18.81
2109080,1,0,1,14.77,16.06,41,22OI7,2X50,ML,STI,0,53,40,NA,18.48
129870,1,1,1,6.59,6.55,52,18OZ00IS,60,G,GEL,0,40,25,NA,9.31
528907,1,0,1,4.39,4.14,90,20OI0,25X2,ST,KOM,0,53,14,NA,8.13
560194,1,0,1,13.66,10.03,90,13OX06,1000,ML,LOE,0,52,127,NA,21.6


In [64]:
# group, content, unit, pharmForm, campaignIndex
train = lapply(train, function(col) {
   if (!inherits(col, "character"))
       return (col)
    
    unclass(factor(col))
})
train = as.data.frame(train)

In [68]:
train = as.matrix(train)

In [72]:
#model = xgboost(data = train, label = train_labels$order,
#                max.depth = 2, eta = 1, nthread = 2, nround = 2, objective = "binary:logistic")
model = xgboost(data = as.matrix(train), label = train_labels$order, max.depth = 2, eta = 1, nthread = 2,
                nround = 2, objective = "binary:logistic")


ERROR: Error in xgb.DMatrix(data, label = label, missing = missing): [20:26:08] amalgamation/../dmlc-core/src/io/local_filesys.cc:66: LocalFileSystem.GetPathInfo  1 Error:No such file or directory
